# Project 1: Using PostgreSQL + Pandas for Data Management & Analysis 
Group member:
1. Hiep Vo Dang

## Part 1: Introduction

For my Project 1, I have chosen to work with the "ATP Tennis Rankings, Results, and Stats" dataset, which is forked from the repository maintained by Jeff Sackmann (https://github.com/JeffSackmann/tennis_atp#atp-tennis-rankings-results-and-stats).

This dataset comprises detailed records of professional men's tennis matches, including individual player stats and match outcomes. From the vast dataset, I've specifically leveraged the files matching the pattern "atp_matches_yyyy.csv", intending to normalize this data into final fact tables and their associated dimension tables. 

My research primarily revolves around two core questions concerning the nuances of professional tennis. 

- First, I aim to determine if there's a correlation between a player's height and their serving prowess, essentially exploring whether taller players usually serve better. 
- Second, I seek to understand the impact of a successful first serve; once a player gets their first serve in, how likely are they to win the point? 

Through this data exploration, I aim to discern patterns and relationships that might provide insights into the intricate dynamics of tennis gameplay, particularly emphasizing the pivotal role of serving.

## Part 2: Data Summary

The dataset houses primary ATP player details, historical rankings, match outcomes, and statistical insights.

The author of this dataset has create many files to ease the use of the dataset, including:
- Player File: Attributes include `player_id`, `first_name`, `last_name`, `hand`, `birth_date`, `country_code`, and `height` (measured in cm).
- Ranking Files: Columns encompass `ranking_date`, `ranking`, `player_id`, and `ranking_points` (when available). ATP rankings are mostly comprehensive from 1985 onwards, with an absence in 1982 and sporadic entries from 1973-1984.
- Match Details: Each season typically has up to three files:
    + Main draw matches at the tour-level (e.g., `atp_matches_2014.csv`)
    + Tour-level qualifying and main-draw matches for challengers
    + Matches at the futures level.

For the purpose of this project, which necessitates database normalization, I have narrowed down my focus to <u>only</u> files that provide statistical data on main draw matches at the tour level. These files can be identified by their naming convention, `atp_matches_yyyy.csv`, where `yyyy` represents the specific year.

It's essential to note that while the overarching dataset does contain tables like `atp_players` and `atp_rankings`, <u>I have intentionally chosen not to interact with them</u>. The rationale behind this decision is to simulate a real-world scenario where I might only have access to the raw form of data, in this case, match statistics. My aim is to conduct the normalization process using this raw data as my sole data source in order to meet the requirements of this project.

Given the availability of data within the ATP dataset, there are specific constraints in terms of the years I have chosen to focus on for this project. While the ATP rankings are largely comprehensive starting from 1985 up to the present day, the detailed statistics for tour-level matches only commence from 1991 onwards. Consequently, to ensure that the data I work with is both relevant and adequately detailed, I have decided to utilize files that range from `atp_matches_1991.csv` through to `atp_matches_2023.csv`. This range provides a substantial time span, ensuring a rich dataset while also aligning with the availability of detailed match statistics.

## Part 3: Data Management using PostgreSQL

In order to get the data seemlessly from raw csv files hosted in a Github repository to my PostgreSQL database, I will develop an <b><u>ETL pipeline</u></b> (Extract, Transform and Load). The database is pre-created by the following script:

```
-- tournaments table
create table if not exists "tournaments" (
    "tourney_id"        varchar(255)    primary key
    , "tourney_name"    varchar(255)    not null
    , "tourney_level"   varchar(255)
    , "tourney_date"    date
    , "surface"         varchar(255)
    , "draw_size"       integer
);

-- players table
create table if not exists "players" (
    "player_id"         varchar(15)     primary key
    , "name"            varchar(255)    not null
    , "hand"            varchar(5)      check ("hand" in ('R', 'L', 'A', 'U'))
    , "height"          float
    , "ioc"             varchar(5)      not null
    , "birth_year"      integer
);

-- matches table with surrogate key
create table if not exists "matches" (
    "match_id"          serial          primary key
    , "season"          integer         not null
    , "tourney_id"      varchar(255)    not null references "tournaments" ("tourney_id") on delete cascade
    , "match_num"       integer         not null
    , "winner_id"       varchar(15)     not null references "players" ("player_id") on delete cascade
    , "loser_id"        varchar(15)     not null references "players" ("player_id") on delete cascade
    , "score"           varchar(255)
    , "best_of"         integer
    , "round"           varchar(50)
    , "minutes"         integer
    , "winner_aces"            integer
    , "winner_double_faults"   integer
    , "winner_serve_points"    integer
    , "winner_first_serve_in"  integer
    , "winner_first_serve_won" integer
    , "winner_second_serve_won" integer
    , "winner_service_games"   integer
    , "winner_break_points_saved" integer
    , "winner_break_points_faced" integer
    , "loser_aces"            integer
    , "loser_double_faults"   integer
    , "loser_serve_points"    integer
    , "loser_first_serve_in"  integer
    , "loser_first_serve_won" integer
    , "loser_second_serve_won" integer
    , "loser_service_games"   integer
    , "loser_break_points_saved" integer
    , "loser_break_points_faced" integer
    , unique("season", "tourney_id", "match_num")   -- ensures combination is unique
);

-- rankings table
create table if not exists "rankings" (
    "ranking_id"        serial          primary key
    , "season"          integer         not null
    , "tourney_id"      varchar(255)    not null references "tournaments" ("tourney_id") on delete cascade
    , "player_id"       varchar(15)     not null references "players" ("player_id") on delete cascade
    , "rank"            float
    , "points"          float
    , unique("season", "tourney_id", "player_id")   -- ensures combination of season and player_id is unique
);

```

The resulting Entity Relation Diagram:

In [ ]:
from IPython import display
display.Image("")